# Graph Neural Networks with Pytorch

Original Paper: https://arxiv.org/abs/1706.02216    
Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/reddit.py

In [ ]:
import os
import sys

import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.datasets import Reddit
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv

sys.path.append('./')
from utils import *
logger = make_logger(name='graphsage_logger')

In [ ]:
# Load Reddit Dataset
path = os.path.join(os.getcwd(), '..', 'data', 'Reddit')
dataset = Reddit(path)
data = dataset[0]

In [ ]:
# Data 확인
# Nodes: 232965, Node Features: 602
logger.info(f"Node Feature Matrix Info: # Nodes: {data.x.shape[0]}, # Node Features: {data.x.shape[1]}")

# Edge Index
# Graph Connectivity- COO (2, num_edges)형태의 그래프 연결 = (2, 114,615,892=1.14억)
logger.info(f"Edge Index Shape: {data.edge_index.shape}")
logger.info(f"Edge Weight: {data.edge_attr}")

# train_mask는 훈련할 nodes을 나타낸다. (153431 nodes)
print(data.train_mask.sum().item())

In [ ]:
# Define Sampler
train_loader = NeighborSampler(
    data.edge_index, node_idx=data.train_mask,
    sizes=[25, 10], batch_size=1024, shuffle=True, num_workers=12)

subgraph_loader = NeighborSampler(
    data.edge_index, node_idx=None,
    sizes=[-1], batch_size=1024, shuffle=False, num_workers=12)

### NeighborSampler Size
sizes=[25, 10]는 Neighbor Sampling에서 사용할 k-hop 크기를 나타낸다.        
sizes 리스트의 첫 번째 값인 25는 1-hop neighbor와 2-hop neighbor까지 사용하여 샘플링할 개수를 의미한다. 이는 중심 노드와 인접한 노드, 그리고 이들의 이웃 노드까지 모두를 포함하여 최대 25개까지 샘플링한다는 것을 뜻한다. 즉, 이 값은 한 번에 샘플링할 이웃 노드의 최대 수를 결정하는 것이다.    
두 번째 sizes 값인 10은 샘플링할 이웃 노드의 개수를 의미한다. 이 값은 첫 번째 값과는 다르게 1-hop neighbor에 대해서만 샘플링을 하며, 2-hop neighbor는 사용하지 않는다. 따라서 이 값은 첫 번째 값보다 작다.    
    
예를 들어, 1-hop neighbor가 A, B, C, D, E, F인 노드 u에 대해 첫 번째 값 25와 두 번째 값 10을 가진 sizes를 사용하면 다음과 같은 샘플링이 이루어진다.    
1) 1-hop neighbor에서 25개 노드를 샘플링한다. 예를 들어, A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y가 선택됩니다.    
     1-2) 2-hop neighbor에서 25개 노드를 샘플링한다. 예를 들어, 2-hop neighbor가 a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y일 때, A와 이웃인 a, b, c, d, e와 B와 이웃인 f, g, h, i, j를 선택한다.    
2) 1-hop neighbor에서 10개 노드를 샘플링한다. 예를 들어, A, B, C, D, E, F, G, H, I, J가 선택됩니다.
즉, 첫 번째 값은 1-hop neighbor와 2-hop neighbor까지 모두 사용하여 샘플링하며, 두 번째 값은 1-hop neighbor에 대해서만 샘플링한다. 이는 모델의 복잡도를 조절할 수 있는 하이퍼파라미터이다.    
이렇게 여러 개의 k-hop 크기를 사용하여 샘플링하는 것은 더 넓은 범위의 이웃 노드 정보를 포함할 수 있도록 하여 모델 성능을 높일 수 있다.        
sizes는 하나의 리스트로 구성되어 있으며, 리스트 내의 값이 클수록 더 많은 이웃 노드 정보를 사용하여 샘플링한다.    

### num_workers 
num_workers는 NeighborSampler가 데이터를 로딩할 때 사용할 worker(thread)의 수를 의미한다. 즉, 병렬적인 데이터 로딩을 위한 파라미터이다. 이 값을 높일수록 데이터 로딩 속도가 빨라지지만, 시스템 리소스를 많이 사용하게 되어 메모리나 CPU 사용량 등을 고려하여 적절한 값을 선택해야 한다. 기본값은 1이며, 사용 가능한 하드웨어 리소스에 따라 값을 조정할 수 있습니다.

### subgraph_loader와 train_loader
train_loader는 훈련 데이터셋을 위한 Neighbor Sampling을 수행한다. 이 때 node_idx 인자에는 훈련 데이터셋에서 사용될 노드 인덱스를 전달한다.     
반면에 subgraph_loader는 일반적으로 테스트 데이터셋에서 사용된다. node_idx 인자에는 None 값을 전달한다. sizes 인자는 -1로 설정하여 원래 그래프 전체를 사용하지 않고, 부분 그래프(subgraph)를 샘플링한다. 이렇게 샘플링된 부분 그래프는 전체 그래프의 근사값(approximation)으로 사용된다.

In [ ]:
batch_size, n_id, adjs = next(iter(train_loader))

logger.info(f"Current Batch Size: {batch_size}")
logger.info(f"현재 Subgraph에서 사용된 모든 node id의 개수: {n_id.shape[0]}")
logger.info(f"Layer의 수: {len(adjs)}")

A = adjs[1].size[0] - batch_size
B = adjs[0].size[0] - A - batch_size

logger.info(f"진행 방향: {B}개의 2-hop neighbors ->"
            f"{A}개의 1-hop neighbors -> {batch_size}개의 Head Nodes")

1. batch_size : 현재 batch size를 의미함 (integer)    
next(iter(train_loader))를 호출하면, batch_size에 해당하는 개수의 샘플들을 반환한다. 이때 반환된 데이터는 이전에 NeighborSampler에서 지정한 sizes 파라미터에 따라 다양한 크기의 서브그래프(subgraph)를 형성한다. 따라서 n_id는 현재 배치에 있는 모든 노드 ID를 담고 있으며, adjs는 현재 배치에 대해 형성된 서브그래프의 리스트이다. 이 서브그래프들은 이후 모델 학습에서 사용된다.

2. n_id : 이번 Subgraph에서 사용된 모든 node id.    
batch_size개의 Row를 예측하기 위해서는 1차 이웃 node A개가 필요하고, 1차 이웃 node A개를 위해서는 2차 이웃 node B개가 필요.     
n_id.shape = batch_size + A + B

3. adjs : 아래와 같이 Layer의 수가 2개이면 adjs는 길이 2의 List가 된다.(subgraph의 리스트)    
head node가 있고 1-hop neighbors와 2-hop neighbors가 있다고 할 때, adjs[0]은 1번째 레이어(1-hop 이웃)와 관련된 정보를 나타내며, head node와 1-hop 이웃들의 관계를 설명한다.     
adjs[1]은 2번째 레이어(2-hop 이웃)와 관련된 정보를 나타내며, 1-hop 이웃들과 2-hop 이웃들 간의 관계를 설명한다. 

4. 각 리스트에는 (edge_index, e_id, size) 튜플이 들어있다.    
edge_index: source -> target nodes를 기록한 bipartite edges    
e_id: 위 edge_index에 들어있는 index가 Full Graph에서 갖는 node id    
size: 위 edge_index에 들어있는 node의 수를 튜플로 나타낸 것으로 head -> 1-hop 관계를 예시로 들면, head node의 수가 a개, 1-hop node의 수가 b개라고 했을 때 size = (a+b, a). 또한 target node의 경우 source nodes의 리스트의 시작 부분에 포함되어 있어 skip-connections나 self-loops를 쉽게 사용할 수 있게 되어 있다.   
   
4-1. bipartite edges    
Bipartite edge는 두 개의 독립된 노드 집합 간의 엣지를 의미한다. 예를 들어, 영화 추천 시스템에서 사용되는 경우, 한 집합은 사용자 노드이고 다른 집합은 영화 노드이다. 이 때, 사용자와 영화 간의 평점을 나타내는 엣지는 bipartite edge가 된다. 일반적으로, bipartite edge는 각 집합 내에서는 연결되어 있지 않으며 다른 집합 내의 노드와만 연결되어 있다.     
    
4-2. edge    
edge란 그래프에서 노드들 간의 연결을 의미하는 선이다. edge는 노드들 간의 상호작용을 모델링하는 데 사용된다. 예를 들어, 소셜 네트워크에서 두 사람 간의 친구 관계, 도시 간의 도로, 단어 간의 유사도 등을 edge로 모델링할 수 있다. 따라서 edge는 그래프 신경망에서 정보 전달과 모델의 예측을 수행하는 중요한 역할을 한다.    
    
    
5. edge_attr, edge_index?   
edge_index는 각 엣지의 시작 노드와 끝 노드의 인덱스를 나타내는 텐서다. 예를 들어, (2, 4)가 edge_index에 있다면, 2번 노드와 4번 노드 사이에 엣지가 존재한다는 것을 의미한다. edge_attr은 엣지의 특성 값을 나타내는 텐서이며, 예를 들어 엣지의 가중치나 거리 등을 나타낼 수 있다. 따라서 edge_index와 edge_attr은 서로 다른 정보를 담고 있는 텐서다.

In [ ]:
# Define Model
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(SAGE, self).__init__()

        self.num_layers = 2

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes 는 항상 먼저 placed된다.
            x = self.convs[i]((x, x_target), edge_index)

            # 마지막 Layer는 Dropout을 적용하지 않는다.
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        return x.log_softmax(dim=-1)

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')

        # # 모든(All) 사용 가능한 edges를 사용하여 노드 표현을 계층별로 계산한다. 
        # 이는 각 batch의 final representations을 즉시 계산하는 것과 대조적으로 
        # 더 빠른 계산으로 이어진다.
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in subgraph_loader:
                edge_index, _, size = adj.to(device)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x.cpu())

                pbar.update(batch_size)

            x_all = torch.cat(xs, dim=0)

        pbar.close()
        return x_all

SAGE 클래스는 PyTorch의 nn.Module 클래스를 상속받고 있다. __init__ 메서드에서는 SAGE 모델의 레이어들을 정의한다.    
    
in_channels : 입력 feature의 차원 수.     
hidden_channels : hidden feature의 차원 수    
out_channels : 출력 feature의 차원 수    
이 예제에서는 두 개의 레이어를 사용하고 있으며, 두 번째 레이어의 출력 feature의 차원 수는 out_channels로 정의된다.    

self.convs는 SAGEConv 레이어를 저장하는 PyTorch의 nn.ModuleList 객체다. nn.ModuleList는 nn.Module 객체를 저장하는 파이썬 리스트와 비슷한 객체다.    
forward 메서드는 모델의 forward propagation을 수행한다. 입력으로 x와 adjs를 받고, x는 그래프의 노드 feature 행렬이며, adjs는 그래프의 인접 리스트다.    

adjs는 다음과 같은 형태의 튜플의 리스트이다:    
edge_index: 그래프의 에지 인덱스를 나타내는 LongTensor
e_id: 사용되지 않음
size: 각 노드 수를 담은 LongTensor
각 레이어에서는 SAGEConv 레이어를 사용하고 있다. 레이어의 입력으로는 (x, x_target)이 사용된다. x는 모든 노드의 feature이고, x_target은 현재 샘플링된 노드의 feature이다. 이러한 방식으로 모델은 그래프의 전체 노드 feature 행렬을 가져오는 대신, 현재 샘플링된 노드들의 feature만을 사용하여 학습한다.

각 레이어에서 마지막에 relu 함수를 거친 뒤 dropout이 적용된다. 마지막 레이어에서는 log_softmax 함수를 사용하여 출력을 생성한다.    
    
추론 함수는 다음과 같다 :    
주어진 데이터 x_all에 대해서, num_layers 개수만큼 SAGEConv 레이어를 통해 embedding vector를 계산한다. 이를 위해, subgraph_loader에서 가져온 각 배치(batch)에 대해서 subgraph를 생성하고, 해당 subgraph에서 x_all에서 선택한 노드에 대한 인접 노드들의 embedding vector를 사용하여 새로운 embedding vector를 계산한다.    
    
반복문이 돌면서 계산한 embedding vector는 x_all에 이어 붙여, 다음 SAGEConv 레이어에 대한 입력으로 사용된다. 이렇게 여러 레이어를 거쳐 최종 embedding vector를 계산한다.    
    
이 과정에서 tqdm 모듈을 사용하여 계산 진행 상황을 표시하고, 계산이 완료되면 최종 embedding vector를 반환한다.    

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGE(dataset.num_features, 256, dataset.num_classes)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

x = data.x.to(device)
y = data.y.squeeze().to(device)

#### PyTorch를 사용하여 GraphSAGE 모델을 정의하고, 데이터셋을 로드하고, 모델을 학습하기 위한 optimizer를 설정   
        
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'): GPU가 사용 가능한 경우 GPU를, 그렇지 않은 경우 CPU를 사용하여 모델 학습을 진행.    
model = SAGE(dataset.num_features, 256, dataset.num_classes): GraphSAGE 모델을 정의. 입력 특성의 크기는 dataset.num_features이고, 은닉층의 크기는 256이며, 출력의 크기는 클래스 수(dataset.num_classes)와 같다.    
model = model.to(device): 모델을 지정된 device(GPU 또는 CPU)로 이동한다.    
optimizer = torch.optim.Adam(model.parameters(), lr=0.01): 모델의 파라미터를 최적화하기 위해 Adam optimizer를 사용하며, 학습률은 0.01로 설정된다.    
x = data.x.to(device): 입력 데이터의 특성을 GPU 또는 CPU로 이동한다.    
y = data.y.squeeze().to(device): 입력 데이터의 레이블을 GPU 또는 CPU로 이동한다. 레이블은 1차원 벡터로 변환되며, squeeze() 메서드를 사용하여 크기가 1인 차원을 제거한다.    

In [ ]:
# 학습
def train(epoch):
    model.train()

    pbar = tqdm(total=int(data.train_mask.sum()))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = 0
    for batch_size, n_id, adjs in train_loader:
        # `adjs` 는 `(edge_index, e_id, size)` tuples의 list를 가지고 있다.
        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out = model(x[n_id], adjs)
        loss = F.nll_loss(out, y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        pbar.update(batch_size)

    pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / int(data.train_mask.sum())
    return loss, approx_acc


########################### 모델 평가
@torch.no_grad()
def test():
    model.eval()
    out = model.inference(x)

    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    results = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        results += [int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]
    return results

 model.train()은 모델을 학습 모드로 설정한다. pbar는 tqdm 라이브러리를 사용하여 현재 학습 진행 상황을 시각화하며, total_loss는 현재까지 누적된 loss 값, total_correct는 현재까지 누적된 정확도 값이다.    
    
train_loader에서는 NeighborSampler를 사용하여 주어진 노드의 이웃 노드들을 샘플링하고, 이를 이용하여 mini-batch를 구성한다. 구성된 mini-batch는 batch_size, n_id, adjs로 구성되어 있습니다.    
    
adjs는 train_loader에서 샘플링된 mini-batch 내의 모든 노드에 대한 이웃 정보를 담고 있는 리스트이다. 이 리스트의 각 요소는 (edge_index, e_id, size)의 튜플이다. edge_index는 샘플링된 mini-batch 내의 노드들 간의 edge 정보를 담고 있으며, size는 각 노드의 이웃 노드들의 개수를 담고 있다.    
    
optimizer.zero_grad()를 호출하여 optimizer의 gradient를 0으로 초기화하고, 모델의 입력으로 mini-batch 내의 노드의 feature와 샘플링된 adjs 정보를 입력으로 넣어 출력을 계산한다. 출력값 out은 softmax 함수를 거쳐 클래스별 확률 값을 가지는 tensor이다. 이 tensor를 이용하여 NLLLoss를 계산하고 backward하여 gradient를 계산한다. 이후 optimizer를 이용하여 모델의 weight를 업데이트한다.    
    
마지막으로 total_loss와 total_correct에 각각 loss와 정확도 값을 더한다. total_loss는 mini-batch의 개수로 나누어 loss의 평균값을 구하고, total_correct는 학습 데이터셋 내에서 모델이 맞힌 레이블의 개수를 학습 데이터셋의 크기로 나누어 정확도를 구한다.    
         
@torch.no_grad() 데코레이터는 해당 함수 내에서의 모든 계산이 경사도를 계산하지 않기 위해 with 구문 대신 사용되며, 이 함수 내에서의 모든 텐서 연산이 추적되지 않는다.        
    
이후에는 모델을 평가하기 위해 필요한 준비 작업들이 이루어진다. model.eval()은 모델을 평가 모드로 설정하며, model.inference(x)는 모든 노드를 대상으로 한번에 계산을 수행하는 inference 함수를 호출한다. 이 함수는 앞서 살펴본 대로 subgraph sampling을 수행하여 모든 노드를 대상으로 한 번에 계산한다.    
    
계산된 out 텐서에서는 argmax를 통해 가장 큰 값을 가지는 클래스를 예측값으로 설정한다. 그리고 이 예측값과 정답값 y_true를 비교하여 모델의 성능을 측정한다.    
    
마지막으로, 성능 측정을 위해 data.train_mask, data.val_mask, data.test_mask를 순회하며 예측 결과를 저장하고, 결과를 반환한다.

In [ ]:
for epoch in range(1, 3):
    loss, acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
    # train_acc, val_acc, test_acc = test()
    # print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

#### train 함수를 이용해 모델을 학습하고, 학습 결과를 출력

range(1,3)은 1부터 2까지 (3은 포함하지 않음)의 숫자를 갖는 리스트를 만든다. 이는 총 2번의 학습(epoch)을 수행한다는 뜻.

loss, acc = train(epoch)는 train 함수에서 반환된 손실값과 정확도를 loss와 acc에 저장한다.

print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')은 현재 학습 epoch 번호, 손실값, 정확도를 출력하는 코드다.

이 코드에서는 주석 처리되어 있지만, test 함수를 이용하여 학습된 모델을 평가하는 코드를 추가할 수 있다. 주석을 해제하면 각 에폭에서의 학습과 평가 결과를 함께 출력할 수 있다.